Open the mask file --> slice area of interest.

Using scipy.convolve, check neighbours of mask file (in 8 directions) --> output coastal cells

## Importing packages

In [ ]:
from dask.distributed import LocalCluster, Client
import dask.array
import datetime
from datetime import date 
from datetime import datetime
import glob
import numpy as np
from scipy import signal
import scipy.ndimage as ndimage
from scipy.ndimage import convolve
from scipy.ndimage.measurements import label, find_objects
import xarray as xr

## Opening the mask file

In [154]:
# NE coast of Tasmania

ds = xr.open_dataset('/g/data/e14/cp3790/Charuni/static_era5.nc').sel(time=slice('1982', '2018'), longitude=slice(147.0, 149.0), latitude=slice(-40.0, -42.0))
mask = np.where(ds.lsm==0, 0, 1)

In [155]:
mask

array([[[0, 0, 1, 1, 1, 1, 1, 0, 0],
        [0, 0, 0, 1, 1, 1, 1, 0, 0],
        [0, 0, 0, 1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 0, 0]]])

k is an array of weights, with the same number of dimensions as input

In [156]:
k =([[[1, 1, 1],
    [1, 0, 1],
    [1, 1, 1]]])

input array 'mask' is convolved with 'k', mode='constant' which ensures that the input is extended by filling all values beyond the edge with the same constant value

In [157]:
c = convolve(mask, k, mode='constant')

In [170]:
c

array([[[0, 1, 2, 4, 5, 5, 3, 2, 0],
        [0, 1, 4, 6, 8, 8, 5, 3, 0],
        [2, 3, 5, 6, 8, 8, 5, 3, 0],
        [3, 5, 6, 7, 8, 8, 5, 3, 0],
        [5, 8, 8, 8, 8, 8, 5, 3, 0],
        [5, 8, 8, 8, 8, 8, 5, 3, 0],
        [5, 8, 8, 8, 8, 8, 5, 3, 0],
        [5, 8, 8, 8, 8, 8, 5, 3, 0],
        [3, 5, 5, 5, 5, 5, 3, 2, 0]]])

coastal array is 1 when the mask==1 and the sum of neighbouring cells is < 8 (which means at least one cell is neighbouring a 0 (or sea grid cell)

In [158]:
coastal = np.logical_and(mask == 1, c < 8)

In [159]:
coastal.astype(int)

array([[[0, 0, 1, 1, 1, 1, 1, 0, 0],
        [0, 0, 0, 1, 0, 0, 1, 0, 0],
        [0, 0, 0, 1, 0, 0, 1, 0, 0],
        [1, 1, 1, 1, 0, 0, 1, 0, 0],
        [1, 0, 0, 0, 0, 0, 1, 0, 0],
        [1, 0, 0, 0, 0, 0, 1, 0, 0],
        [1, 0, 0, 0, 0, 0, 1, 0, 0],
        [1, 0, 0, 0, 0, 0, 1, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 0, 0]]])

## Anomaly dataset

In [168]:
# surface air temperature anomalies
tasa = xr.open_dataset('/g/data/e14/cp3790/Charuni/surface_air_temp_anomaly.nc').sel(time=slice('1982', '2018'), longitude=slice(147.0, 149.0), latitude=slice(-40.0, -42.0))

In [169]:
tasa

<xarray.Dataset>
Dimensions:    (latitude: 9, longitude: 9, time: 13514)
Coordinates:
  * longitude  (longitude) float32 147.0 147.25 147.5 ... 148.5 148.75 149.0
  * latitude   (latitude) float32 -40.0 -40.25 -40.5 ... -41.5 -41.75 -42.0
  * time       (time) datetime64[ns] 1982-01-01 1982-01-02 ... 2018-12-31
Data variables:
    anomaly    (time, latitude, longitude) float32 ...

In [161]:
files = sorted(glob.glob('/g/data/e14/cp3790/Charuni/MHW-sev/mhw_severity.pc90.*.nc'))
mhw = xr.open_mfdataset(files, combine='by_coords').sel(time=slice('1982', '2018')

In [166]:
# sea surface temperature anomalies 
ssta = mhw.ssta.sel(time=slice('1982', '2018'), lon=slice(147.0, 149.0), lat=slice(-42.0, -40.0))

In [167]:
ssta

<xarray.DataArray 'ssta' (time: 13514, lat: 8, lon: 8)>
dask.array<getitem, shape=(13514, 8, 8), dtype=float32, chunksize=(13514, 8, 8), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 1982-01-01 1982-01-02 ... 2018-12-31
  * lat      (lat) float64 -41.88 -41.62 -41.38 -41.12 ... -40.62 -40.38 -40.12
  * lon      (lon) float64 147.1 147.4 147.6 147.9 148.1 148.4 148.6 148.9
Attributes:
    units:      degrees C
    long_name:  Daily sea surface temperature anomaly